# Задание к занятию «Рекомендации на основе скрытых факторов»
Преподаватель: Борис Шминке
Описание задания:
Что делать
1. Установить implicit;
2. Взять датасет last.fm (урезанный или полный);
3. Разбить датасет на обучающую и тестовую выборки;
4. Построить на обучающей выборке хотя бы две модели из пакета implicit:
    - kNN по косинусной мере
    - ALS
5. Получить рекомендации на тестовой выборке для обученных моделей;
6. Сравнить метрики качества обученных моделей на тестовой выборке с помощью mrec (или иным способом).

Начнем с малого датасета, как минимум потому что на нем быстрее.

In [1]:
import pandas as pd
import numpy as np

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv("lastfm_small.tsv",
    sep="\t",
    header=None,
    names=col_names
)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


заполняем пустые значения и заменим строковые идентификаторы числовыми кодами

In [2]:
data.fillna("None", inplace=True)

data["user_id"] = data["user"].astype("category").cat.codes.copy() 
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy()
data["plays"] = data["total-plays"].astype(np.double)

убираем лишние колонки

In [3]:
data.drop(["artist-name", "artist-mbid", "user", "total-plays"], axis=1, inplace=True)
data.head(5)

,user_id,artist_id,plays
0,0,15530,2137.0
1,0,63468,1099.0
2,0,46857,897.0
3,0,15967,717.0
4,0,48968,706.0


In [4]:
data[data["artist_id"] == 0]

,user_id,artist_id,plays
393800,8053,0,36.0


Посмотрим на наши данные

In [5]:
data.describe()

,user_id,artist_id,plays
count,1000000.000000,1000000.000000,1000000.000000
mean,10231.925996,33677.492236,216.606950
std,5912.022447,19230.330182,604.378024
min,0.000000,0.000000,1.000000
25%,5117.000000,17297.000000,34.000000
50%,10236.000000,34543.000000,94.000000
75%,15346.000000,49487.000000,225.000000
max,20464.000000,66798.000000,135392.000000


разобьём наблюдения на тестовую и обучающую выборки

In [6]:
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [7]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 1
нет в тестовой выборке, но есть в обучающей: 22
всего пользователей: 20465


In [8]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)


/Users/vladimir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 0
нет в тестовой выборке, но есть в обучающей: 0
всего пользователей: 20465


In [10]:
import argparse
import logging
import time

import numpy
import pandas
from scipy.sparse import coo_matrix

from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)

## Найдем похожих артистов методом ALS

Создадим разреженную матрицу для обучения

In [11]:
plays = coo_matrix((train_data['plays'],
                   (train_data['artist_id'],
                    train_data['user_id'])))

user_plays = plays.tocsr()

Создадим модель ALS

In [12]:
model = AlternatingLeastSquares(factors=64, dtype=np.float32)
model_name = model.__class__
model.approximate_recommend = False

Обучим нашу модель (заполним данными матрицу)

In [13]:
model.fit(user_plays)

Выведем ближайших к какому то артисту (проходимся по всем в цикле) артистов (Ограничимся первым десятком, по 5 сравнений)

In [14]:
artists = dict(enumerate(train_data['artist_id']))

to_generate = train_data.groupby('artist_id').size().sort_values(ascending=False)
for artistid in to_generate[:10]:
    artist = artists[artistid]
#     print('')
#     for other, score in model.similar_items(artistid, N=5):
#         print("{}\t{}\t{}\n".format(artist, artists[other], score))

## Создадим методы с метриками

In [15]:
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, precision_score
from math import sqrt
def rmse(predict, truth):   
    return sqrt(mean_squared_error(predict, truth))

In [16]:
def roc_auc(predict, truth):
    
    return roc_auc_score(predict, truth)

In [17]:
def ave_prec(predict, truth):

    return average_precision_score(truth, predict)  

In [18]:
def prec(predict, truth):

    return precision_score(truth, predict)  

In [19]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

## Предскажем артистов методом ALS

Создадим модель ALS

In [20]:
model = AlternatingLeastSquares(factors=128, dtype=np.float32)
model_name = model.__class__
model.approximate_similar_items = False

Обучим нашу модель (заполним данными матрицу)

In [21]:
model.fit(plays)

In [22]:
# sparse_info(model.similarity)

Выведем все предсказанные нами рекомендации артистов для пльзователей (на самом деле список длинный, поэтому строку print закоментил)

In [36]:
predict_artist = []
user_plays = plays.T.tocsr()    
test_unique = test_data['user_id'].unique()
for userid, username in enumerate(test_unique):
    for artistid, score in model.recommend(userid, user_plays):        
        predict_artist.append((userid,artistid, score))  

Приведем полученный результат к датафрейму

In [37]:
predict = pd.DataFrame(predict_artist, columns=["user_id","artist_id","plays"])
predict.head()

,user_id,artist_id,plays
0,0,31400,1.541331
1,0,191,1.501564
2,0,56178,1.491869
3,0,19992,1.371267
4,0,54682,1.303848


In [39]:
predict["plays"].max()

2.0732989311218262

#### Для получения данных с метрик оставим только тех пользователей для которых у нас были данные по артистам и для которых были предсказанны данные по артистам. Что бы получить рейтинги и сравнить их

In [40]:
merged_data = pd.merge(test_data, predict, on=['user_id', 'artist_id'])

In [41]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23416 entries, 0 to 23415
Data columns (total 4 columns):
user_id      23416 non-null int16
artist_id    23416 non-null int32
plays_x      23416 non-null float64
plays_y      23416 non-null float64
dtypes: float64(2), int16(1), int32(1)
memory usage: 686.0 KB


In [42]:
print( 'RMSE: ' + str(rmse(merged_data["plays_y"], merged_data["plays_x"])))

RMSE: 700.3791574253917


Посчитаем среднее в предсказанных данных и в тестовых. Каждое среднее будем считать отдельно, потому что абстрактные цифры предсказаний надо как то нормально интерпретировать :) 

In [43]:
mean_predict = merged_data["plays_y"].mean()
mean_predict

0.9692465669482974

In [44]:
mean_truth = merged_data["plays_x"].mean()
mean_truth 

279.6576699692518

In [45]:
predict = merged_data["plays_y"].apply(lambda x: 1 if x > mean_predict else 0 )
truth = merged_data["plays_x"].apply(lambda x: 1 if x > mean_truth else 0 )

In [46]:
print( 'Average precision: ' + str(ave_prec(predict, truth)))

Average precision: 0.359131793399


In [47]:
print( 'Precision: ' + str(prec(predict, truth)))

Precision: 0.379906046371


In [48]:
print( 'ROC AUC: ' + str(roc_auc(predict, truth)))

ROC AUC: 0.639453903984


## Предскажем артистов методом kNN по косинусной мере

Создадим модель kNN по косинусной мере

In [50]:
model = CosineRecommender()
model_name = model.__class__
model.approximate_similar_items = False

Обучим нашу модель (заполним данными матрицу)

In [51]:
model.fit(plays)

In [52]:
sparse_info(model.similarity)

Размерности матрицы: (66799, 66799)
Ненулевых элементов в матрице: 1216723
Доля ненулевых элементов: 0.00027267906469628805
Среднее значение ненулевых элементов: 0.4442609287484492
Максимальное значение ненулевых элементов: 1.0000000000000044
Минимальное значение ненулевых элементов: 1.3436439542019536e-07


Выведем все предсказанные нами рекомендации артистов для пльзователей (на самом деле список длинный, поэтому строку print закоментил)

In [53]:
predict_artist = []
user_plays = plays.T.tocsr()        
for userid, username in enumerate(test_data['user_id'].unique()):
    for artistid, score in model.recommend(userid, user_plays):        
        predict_artist.append((userid,artistid, score))

Приведем полученный результат к датафрейму

In [54]:
predict = pd.DataFrame(predict_artist, columns=["user_id","artist_id","plays"])
predict.head()

,user_id,artist_id,plays
0,0,15193,878.529741
1,0,21330,835.677714
2,0,13606,789.061561
3,0,3819,767.773949
4,0,11933,761.973803


In [55]:
test_data.head()

,user_id,artist_id,plays
189773,3880,1457,91.0
949647,19436,41045,642.0
525431,10759,7403,32.0
933248,19107,37418,70.0
622607,12739,43646,95.0


#### Для получения данных с метрик оставим только тех пользователей для которых у нас были данные по артистам и для которых были предсказанны данные по артистам. Что бы получить рейтинги и сравнить их

In [56]:
merged_data = pd.merge(test_data, predict, on=['user_id', 'artist_id'])

In [57]:
merged_data.head()

,user_id,artist_id,plays_x,plays_y
0,13449,47021,49.0,80.116542
1,8272,28743,34.0,35.930400
2,5372,52310,432.0,354.910689
3,4028,60159,280.0,609.729634
4,17523,7777,1201.0,1030.284201


In [58]:
print( 'RMSE: ' + str(rmse(merged_data["plays_y"], merged_data["plays_x"])))

RMSE: 1071.499788214627


Посчитаем среднее в предсказанных данных и в тестовых. Каждое среднее будем считать отдельно, потому что абстрактные цифры предсказаний надо как то нормально интерпретировать :) 

In [59]:
mean_predict = merged_data["plays_y"].mean()
mean_predict

653.0739829085107

In [60]:
mean_truth = merged_data["plays_x"].mean()
mean_truth 

315.9959704499664

In [61]:
predict = merged_data["plays_y"].apply(lambda x: 1 if x > mean_predict else 0 )
truth = merged_data["plays_x"].apply(lambda x: 1 if x > mean_truth else 0 )

In [62]:
print( 'Average precision: ' + str(ave_prec(predict, truth)))

Average precision: 0.462870974772


In [63]:
print( 'Precision: ' + str(prec(predict, truth)))

Precision: 0.558064516129


In [64]:
print( 'ROC AUC: ' + str(roc_auc(predict, truth)))

ROC AUC: 0.721415070565
